In [1]:
%load_ext autoreload
%autoreload 2

# Init

In [2]:
import sys
sys.path.insert(0, "../src")

import json
import random
from pathlib import Path
from collections import Counter

from gen.util import read_data, write_jsonl
from feverise import scifact, climatefever, constants

In [3]:
feverp = Path("/users/k21190024/study/fact-check-transfer-learning/scratch/data/fever")
climatefp = Path("/users/k21190024/study/fact-check-transfer-learning/scratch/data/climatefever")
scifactp = Path("/users/k21190024/study/fact-check-transfer-learning/scratch/data/scifact")

climatefdp = Path("/users/k21190024/study/fact-check-transfer-learning/scratch/dumps/feverised-climatefever")
scifactdp = Path("/users/k21190024/study/fact-check-transfer-learning/scratch/dumps/feverised-scifact")

climatefdp.mkdir(exist_ok=True), scifactdp.mkdir(exist_ok=True)

(None, None)

fever_dev_sample, wiki_sample = None, None

fever_dev_sample = read_data(feverp / "train.jsonl")
for p in feverp.joinpath("wiki-pages").iterdir():
    if random.uniform(0, 1) > 0.7:
        wiki_sample = read_data(p)
        break

# SciFact

 - Every claim has only 1 type of label

In [4]:
sf_dev = read_data(scifactp / "claims_dev.jsonl")
sf_test = read_data(scifactp / "claims_test.jsonl")
sf_train = read_data(scifactp / "claims_train.jsonl")
sf_corpus = read_data(scifactp / "corpus.jsonl")

In [5]:
sf_dev_f = scifact.feverise_claims(sf_dev)
sf_train_f = scifact.feverise_claims(sf_train)

sf_corpus_f = scifact.feverise_corpus(sf_corpus)

In [6]:
scifactdp.joinpath("wiki-pages").mkdir(exist_ok=True)
write_jsonl(scifactdp / "dev.jsonl", sf_dev_f)
write_jsonl(scifactdp / "train.jsonl", sf_train_f)
write_jsonl(scifactdp / "test.jsonl", sf_test)
write_jsonl(scifactdp / "wiki-pages" / "wiki-001.jsonl", sf_corpus_f)

PosixPath('/users/k21190024/study/fact-check-transfer-learning/scratch/dumps/feverised-scifact/wiki-pages/wiki-001.jsonl')

# Climate-FEVER

- Has multiple labels for each sentence for each claim

In [7]:
cf_data = read_data(climatefp / "climatefever.jsonl")
cf_paper_f, cf_assumed_f, cf_corpus_f = climatefever.feverise_climatefever(cf_data)

In [9]:
climatefdp.joinpath("wiki-pages").mkdir(exist_ok=True)
write_jsonl(climatefdp / "paper_all.jsonl", cf_paper_f)
write_jsonl(climatefdp / "assumed_all.jsonl", cf_assumed_f)
write_jsonl(climatefdp / "wiki-pages" / "wiki-001.jsonl", cf_corpus_f)

PosixPath('/users/k21190024/study/fact-check-transfer-learning/scratch/dumps/feverised-climatefever/wiki-pages/wiki-001.jsonl')